## **Preprocesamiento**

Instalar y cargar librerías

In [1]:
! pip install plotly
! pip install scikit-learn


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.impute import SimpleImputer 

Funciones

In [3]:
#Función para analizar datos no nulos, tipo de variable, cantidad de registros y columnas
def analisis(x):
    print('Información:')
    
    y=x.info()
    z=x.shape

    print('Cantidad de registros y variables', z)

In [4]:
#Función para calcular cantidad y porcentaje de nulos por variable
def nulos(x):
    cant_nulos=x.isnull().sum() 
    porc_nulos=round(x.isnull().sum().sum()/x.size, 4)*100
    print('Cantidad de nulos por variable:') 
    print(cant_nulos)
    print('Porcentaje total de nulos: ', porc_nulos)

In [5]:
#Función para verificar los valores únicos de cada variable
def valores(x):
    for variable in list(x.columns[1:]):
        y = x[variable].unique()
        print(variable)
        print(y)

In [6]:
#Función para pasar nombres de variables en minúscula
def minust(x):
    x.columns = map(str.lower,x.columns)

In [7]:
#Función para convertir registros de una variable en minúscula
def minus(x):
    x=x.str.lower()
    return x

Lectura de datos

In [8]:
df_employee= pd.read_csv('./data/employee_survey_data.csv')
df_general=pd.read_csv('./data/general_data.csv', sep=";")
df_manager=pd.read_csv('./data/manager_survey_data.csv')
df_retirement=pd.read_csv('./data/retirement_info.csv', sep=";")

##### Exploración y limpieza de datos

##### BD employee_survey

In [9]:
df_employee.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [10]:
#Aplicar función de Analisis
analisis(df_employee)
#la BD employee_survey contiene 4 variables, las cuales son el ID del empleado, nivel de satisfación del ambiente laboral, 
#nivel de satisfación laboral, Balance de la vida laboral y personal. 
# En total se tienen 4410 registros de empleados.

Información:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB
Cantidad de registros y variables (4410, 4)


In [11]:
# Convertir variables tipo númericas a categóricas
df_employee=df_employee.astype({'EnvironmentSatisfaction': object, 'JobSatisfaction': object, 'WorkLifeBalance':object})

In [12]:
#Aplicar función de nulos
nulos(df_employee)

Cantidad de nulos por variable:
EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64
Porcentaje total de nulos:  0.47000000000000003


In [13]:
#Dado que se tiene un % de datos nulos muy pequeño para cada varible, se opta por aplicar la estrategia de tratamiento de nulos ffill
# ffill(): copiar el valor previo en la celda donde encuentre un valor nulo
df_employee = df_employee.ffill()

In [14]:
#Aplicar función para ver valores únicos por variable
valores(df_employee)

EnvironmentSatisfaction
[3. 2. 4. 1.]
JobSatisfaction
[4. 2. 1. 3.]
WorkLifeBalance
[2. 4. 1. 3.]


In [15]:
#Aplicar función para convertir títulos en minuscula
minust(df_employee)

##### BD general_data

In [16]:
df_general.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [17]:
#Aplicar función de analisis
analisis(df_general)
#la BD employee_survey contiene 23 variables
# En total se tienen 4410 registros de empleados.

Información:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   int64  
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   int64  
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   int64  
 13  NumCompaniesWorked       4391 non-null   float64
 14  Over18     

In [18]:
#Aplicar función para ver valores únicos por variable
valores(df_general)

BusinessTravel
['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department
['Sales' 'Research & Development' 'Human Resources']
DistanceFromHome
[ 6 10 17  2  8 11 18  1  7 28 14  3  4 16  9  5 20 29 15 13 24 19 22 25
 21 26 27 12 23]
Education
[2 1 4 5 3]
EducationField
['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
EmployeeCount
[1]
EmployeeID
[   1    2    3 ... 4408 4409 4410]
Gender
['Female' 'Male']
JobLevel
[1 4 3 2 5]
JobRole
['Healthcare Representative' 'Research Scientist' 'Sales Executive'
 'Human Resources' 'Research Director' 'Laboratory Technician'
 'Manufacturing Director' 'Sales Representative' 'Manager']
MaritalStatus
['Married' 'Single' 'Divorced']
MonthlyIncome
[131160  41890 193280 ...  37020  23980  54680]
NumCompaniesWorked
[ 1.  0.  3.  4.  2.  7.  9.  5.  6.  8. nan]
Over18
['Y']
PercentSalaryHike
[11 23 15 12 13 20 22 21 17 14 16 18 19 24 25]
StandardHours
[8]
StockOptionLevel
[0 1 3 2]
TotalWorkingYears
[ 1.  6.  5. 13.

In [19]:
#Se detectó que las variables 'EmployeeCount','Over18','StandardHours' están compuestas por un valor único, por tanto, se procede a eliminarlas
df_general=df_general.drop(['EmployeeCount','Over18','StandardHours'], axis=1)

In [20]:
#Aplicar función nulos para identificar la cantidad y porcentaje de estos por cada variable
nulos(df_general)

Cantidad de nulos por variable:
Age                         0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
PercentSalaryHike           0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64
Porcentaje total de nulos:  0.03


In [21]:
# Para las variables de 'TotalWorkingYears' y 'NumCompaniesWorked' se detectaron nulos pero en una porpoción pequeña (inferior del 10%)
#Definir la forma de imputar nulos
imputar = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
# imputar datos faltantes
df_general.iloc[:,:] = imputar.fit_transform(df_general.iloc[:,:]) 

In [22]:
# Convertir variables tipo númericas a categóricas
df_general=df_general.astype({'Education': object, 'JobLevel': object, 'StockOptionLevel':object})
# Convertir variables tipo float a enteras
df_general=df_general.astype({'NumCompaniesWorked': int})

In [23]:
#Aplicar función para convertir en minúscula los registros de las variables tipo string 
df_general['BusinessTravel']=minus(df_general['BusinessTravel'])
df_general['Department']=minus(df_general['Department'])
df_general['EducationField']=minus(df_general['EducationField'])
df_general['Gender']=minus(df_general['Gender'])
df_general['JobRole']=minus(df_general['JobRole'])
df_general['MaritalStatus']=minus(df_general['MaritalStatus'])
#Aplicar función para convertir nombres de variables en minúscula
minust(df_general)

##### BD Manager Survey

In [24]:
df_manager.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [25]:
#Aplicar función de analisis
analisis(df_manager)
#la BD employee_survey contiene 3 variables
#En total se tienen 4410 registros de empleados.

Información:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB
Cantidad de registros y variables (4410, 3)


In [26]:
#Aplicar función para ver valores únicos por variable
valores(df_manager)

JobInvolvement
[3 2 1 4]
PerformanceRating
[3 4]


In [27]:
# Convertir variables tipo númericas a categóricas
df_manager=df_manager.astype({'JobInvolvement': object, 'PerformanceRating': object})

In [28]:
#Aplicar función para identificar la cantidad y porcentaje de nulos por cada variable
nulos(df_manager)
#No tenemos presencia de nulos

Cantidad de nulos por variable:
EmployeeID           0
JobInvolvement       0
PerformanceRating    0
dtype: int64
Porcentaje total de nulos:  0.0


In [29]:
#Aplicar función para convertir nombres de variables en minúscula
minust(df_manager)

##### BD Retirement info

In [30]:
df_retirement.head()

,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,2,Yes,15/10/2016,Resignation,Others
1,7,Yes,20/07/2016,Resignation,Stress
2,14,Yes,08/11/2016,Resignation,Others
3,29,Yes,15/10/2016,Resignation,Others
4,31,Yes,08/02/2016,Resignation,Salary


In [31]:
#Aplicar función de análisis
analisis(df_retirement)
#la BD employee_survey contiene 5 variables
# En total se tienen 711 registros de empleados.

Información:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  641 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB
Cantidad de registros y variables (711, 5)


In [32]:
#Reemplazar nulos de columna de razón de retiro por 'no aplica', ya que estos empleados corresponden a los despedidos
df_retirement['resignationReason'] = df_retirement['resignationReason'].replace(np.nan, 'no aplica')

In [33]:
#Aplicar función pra ver valores únicos por variable
valores(df_retirement)

Attrition
['Yes']
retirementDate
['15/10/2016' '20/07/2016' '08/11/2016' '08/02/2016' '18/06/2016'
 '29/04/2016' '15/12/2016' '18/01/2016' '23/08/2016' '17/02/2016'
 '23/02/2016' '07/04/2016' '08/10/2016' '12/08/2016' '14/10/2016'
 '26/07/2016' '02/05/2016' '29/07/2016' '10/01/2016' '29/02/2016'
 '25/03/2016' '30/06/2016' '16/11/2016' '13/02/2016' '05/07/2016'
 '22/02/2016' '03/01/2016' '05/03/2016' '04/09/2016' '25/01/2016'
 '01/06/2016' '22/06/2016' '28/10/2016' '23/06/2016' '30/05/2016'
 '14/07/2016' '17/08/2016' '28/01/2016' '19/01/2016' '06/04/2016'
 '06/03/2016' '08/12/2016' '01/09/2016' '11/09/2016' '09/06/2016'
 '30/09/2016' '17/01/2016' '26/08/2016' '13/03/2016' '19/06/2016'
 '18/05/2016' '09/03/2016' '24/08/2016' '11/06/2016' '19/02/2016'
 '22/04/2016' '11/08/2016' '02/09/2016' '01/04/2016' '26/05/2016'
 '21/07/2016' '26/11/2016' '02/06/2016' '26/06/2016' '25/05/2016'
 '12/12/2016' '22/10/2016' '06/11/2016' '10/09/2016' '01/05/2016'
 '14/11/2016' '13/12/2016' '21/09/2016' '27

In [34]:
#Se detectó que la variable 'Attrition' está compuesta por un valor único, por tanto, se procede a eliminarla
df_retirement=df_retirement.drop(['Attrition'], axis=1)

In [35]:
#Aplicar función para que los registros de las variables tipo string se han convertidos a minúsculas
df_retirement['retirementType']=minus(df_retirement['retirementType'])
df_retirement['resignationReason']=minus(df_retirement['resignationReason'])
#Aplicar función para convertir nombre de las variables en minúscula
minust(df_retirement)

In [36]:
#Convertir variable 'retirementdate' a formato fecha
df_retirement['retirementdate'] = pd.to_datetime(df_retirement['retirementdate'])

C:\Users\Valentina\AppData\Local\Temp\ipykernel_27044\2124130599.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_retirement['retirementdate'] = pd.to_datetime(df_retirement['retirementdate'])


In [37]:
df_retirement

,employeeid,retirementdate,retirementtype,resignationreason
0,2,2016-10-15,resignation,others
1,7,2016-07-20,resignation,stress
2,14,2016-08-11,resignation,others
3,29,2016-10-15,resignation,others
4,31,2016-08-02,resignation,salary
...,...,...,...,...
706,4382,2016-04-18,resignation,salary
707,4387,2016-09-16,resignation,others
708,4389,2016-05-24,resignation,others
709,4392,2016-02-26,fired,no aplica


##### Unión de dfs

In [38]:
df= pd.merge(df_employee,df_general,on="employeeid",how="inner")
df = pd.merge(df,df_manager,on="employeeid",how="inner")
df = pd.merge(df,df_retirement,on="employeeid",how="left")

In [43]:
#La variable 'retirementType' indica el tipo de retiro del empleado, el cual puede ser por despido o renuncia
#Para el trabajo en cuestión, solo interesa lo campos de empleados que hayan renunciado.
#Por tanto, se eliminan los registros de empleados despedidos
df.drop(df[(df['retirementtype'] =='fired')].index, inplace=True)

##### Descargar df

In [44]:
df.to_csv('./data/df.csv', index=False)